# Appendix G — Docker and Containerisation for ML
## *Python for AI/ML: A Complete Learning Journey*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/APP_G_Docker_Containerisation.ipynb)
&nbsp;&nbsp;[![Back to TOC](https://img.shields.io/badge/Back_to-Table_of_Contents-1B3A5C?style=flat-square)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/Python_for_AIML_TOC.ipynb)

---

**Prerequisites:** Chapter 11 (MLOps and Production ML — FastAPI endpoint)  

### Learning Objectives

- Explain why containers solve the 'works on my machine' problem
- Write a `Dockerfile` that packages the Chapter 11 FastAPI salary endpoint
- Build and run a Docker image locally
- Write a `docker-compose.yml` that runs the API alongside an MLflow tracking server
- Understand image layers, caching, and how to keep images small
- Push an image to Docker Hub and understand cloud registry options
- Know when to use Docker vs a cloud function vs a managed endpoint


---

## G.1 — Why Containers?

When you trained your salary model in Chapter 11, it worked perfectly in Colab.
But Colab runs Python 3.11 with specific library versions. Your colleague's laptop
runs Python 3.9. The production server runs 3.12. The FastAPI app that works
locally throws `ImportError` in production.

**A container packages your code together with its entire runtime environment:**
Python version, all libraries, system dependencies, and configuration.
The container runs identically on any machine that has Docker installed.

```
  Without containers:          With containers:
  ┌─────────────────┐          ┌─────────────────────────────────┐
  │  Your code      │          │  Container                       │
  │  (Python 3.11)  │          │  ┌─────────────────┐            │
  ├─────────────────┤          │  │  Your code      │            │
  │  Dev machine    │          │  │  Python 3.11    │            │
  │  (libraries A)  │          │  │  Libraries A    │            │
  └─────────────────┘          │  └─────────────────┘            │
  ≠                            ├─────────────────────────────────┤
  ┌─────────────────┐          │  Docker runtime                  │
  │  Prod server    │          ├─────────────────────────────────┤
  │  (Python 3.12)  │          │  Any OS, any machine             │
  │  (libraries B)  │          └─────────────────────────────────┘
  └─────────────────┘          = identical everywhere
```

**Core Docker concepts:**

- **Image** — a read-only snapshot of a filesystem. Built from a `Dockerfile`.
- **Container** — a running instance of an image. Isolated from the host.
- **Layer** — each instruction in a `Dockerfile` adds a layer. Layers are cached;
  unchanged layers are reused on rebuild, making builds fast.
- **Registry** — a store for images. Docker Hub is the public default;
  AWS ECR, GCP Artifact Registry, and Azure ACR are common private options.


---

## G.2 — Writing a Dockerfile for the Salary API

The Chapter 11 FastAPI endpoint serves salary predictions from the MLflow-registered
model. Below is the production-grade `Dockerfile` for that service.


In [ ]:
# G.2.1 -- Generate all Docker files for the salary API service

import os

os.makedirs('/tmp/salary_api_docker', exist_ok=True)

# ── Dockerfile ────────────────────────────────────────────────────
DOCKERFILE = '''
# ── Stage 1: builder ─────────────────────────────────────────────
# Use a full Python image to install dependencies
FROM python:3.11-slim AS builder

WORKDIR /app

# Install build tools (needed for some Python packages)
RUN apt-get update && apt-get install -y --no-install-recommends \\
    build-essential \\
    && rm -rf /var/lib/apt/lists/*

# Copy and install dependencies first (layer caching)
# If requirements.txt doesn't change, this layer is reused on rebuild
COPY requirements.txt .
RUN pip install --no-cache-dir --prefix=/install -r requirements.txt

# ── Stage 2: runtime ─────────────────────────────────────────────
# Use a minimal image for the final container
FROM python:3.11-slim

WORKDIR /app

# Copy installed packages from builder stage
COPY --from=builder /install /usr/local

# Copy application code
COPY salary_api.py .

# Create non-root user for security best practice
RUN useradd --no-create-home --shell /bin/false appuser
USER appuser

# Document the port (does not actually publish it)
EXPOSE 8000

# Health check: Docker will restart the container if this fails
HEALTHCHECK --interval=30s --timeout=10s --start-period=15s --retries=3 \\
    CMD python -c "import urllib.request; urllib.request.urlopen('http://localhost:8000/health')"

# Start the FastAPI server
# --workers 2: two worker processes for concurrent requests
CMD ["uvicorn", "salary_api:app", "--host", "0.0.0.0", "--port", "8000", "--workers", "2"]
'''

# ── requirements.txt ──────────────────────────────────────────────
REQUIREMENTS = '''
fastapi==0.115.0
uvicorn[standard]==0.32.0
pydantic==2.9.2
mlflow==2.17.0
scikit-learn==1.5.2
numpy==1.26.4
pandas==2.2.3
httpx==0.27.2
'''

# ── .dockerignore ─────────────────────────────────────────────────
DOCKERIGNORE = '''
# Never copy these into the image
__pycache__/
*.py[cod]
.env
.git/
*.ipynb
mlruns/
mlflow.db
data/
models/
tests/
*.md
Dockerfile*
docker-compose*.yml
'''

for fname, content in [
    ('Dockerfile',       DOCKERFILE),
    ('requirements.txt', REQUIREMENTS),
    ('.dockerignore',    DOCKERIGNORE),
]:
    path = f'/tmp/salary_api_docker/{fname}'
    with open(path, 'w') as f:
        f.write(content.lstrip('\n'))
    print(f'Written: {fname}')

print()
print('Key Dockerfile patterns used:')
print('  Multi-stage build:  builder stage installs deps; runtime stage is minimal')
print('  Layer caching:      COPY requirements.txt before COPY . (slow deps cached)')
print('  Non-root user:      run as appuser, not root (security best practice)')
print('  HEALTHCHECK:        Docker auto-restarts unhealthy containers')
print('  .dockerignore:      keeps image small by excluding dev files')


---

## G.3 — docker-compose: API + MLflow Tracking Server

A real ML deployment has multiple services: the prediction API,
the MLflow tracking server, and a database for MLflow metadata.
`docker-compose` orchestrates multiple containers as a single application.

**Service architecture:**

```
  ┌──────────────────────────────────────────────────────┐
  │  docker-compose network                               │
  │                                                       │
  │  ┌──────────────────┐    ┌────────────────────────┐  │
  │  │  salary-api      │    │  mlflow-server          │  │
  │  │  :8000           │◄──►│  :5000                  │  │
  │  │  FastAPI +       │    │  Tracking UI            │  │
  │  │  loaded model    │    │  Model Registry         │  │
  │  └──────────────────┘    └────────────────────────┘  │
  │           │                         │                  │
  │  ┌────────▼─────────────────────────▼──────────────┐  │
  │  │  shared volume: /mlruns  (model artefacts)       │  │
  │  └─────────────────────────────────────────────────┘  │
  └──────────────────────────────────────────────────────┘
       Port 8000 and 5000 exposed to host machine
```


In [ ]:
# G.3.1 -- Generate docker-compose.yml

DOCKER_COMPOSE = '''
# docker-compose.yml
# Runs the salary prediction API alongside the MLflow tracking server
# Usage: docker-compose up --build

version: "3.9"

services:

  # ── MLflow Tracking Server ───────────────────────────────────────
  mlflow-server:
    image: python:3.11-slim
    command: >
      bash -c "pip install mlflow==2.17.0 -q &&
               mlflow server
               --host 0.0.0.0
               --port 5000
               --backend-store-uri sqlite:////mlruns/mlflow.db
               --default-artifact-root /mlruns/artefacts"
    ports:
      - "5000:5000"              # MLflow UI at http://localhost:5000
    volumes:
      - mlruns_data:/mlruns      # persist runs and models between restarts
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:5000/health"]
      interval: 30s
      timeout: 10s
      retries: 3

  # ── Salary Prediction API ────────────────────────────────────────
  salary-api:
    build:
      context: .                 # build from Dockerfile in current dir
      dockerfile: Dockerfile
    ports:
      - "8000:8000"              # API at http://localhost:8000
      #                            Swagger docs at http://localhost:8000/docs
    environment:
      # Model URI: load the Production-stage model from the MLflow registry
      MODEL_URI: "models:/so2025_salary_predictor/Production"
      MLFLOW_TRACKING_URI: "http://mlflow-server:5000"
    volumes:
      - mlruns_data:/mlruns      # shared with mlflow-server
    depends_on:
      mlflow-server:
        condition: service_healthy
    restart: unless-stopped      # auto-restart on crash

volumes:
  mlruns_data:                   # named volume persists between docker-compose up/down
'''

with open('/tmp/salary_api_docker/docker-compose.yml', 'w') as f:
    f.write(DOCKER_COMPOSE.lstrip('\n'))
print('Written: docker-compose.yml')

print()
print('To run the full stack:')
print('  cd /tmp/salary_api_docker')
print('  docker-compose up --build')
print()
print('Then access:')
print('  API:         http://localhost:8000')
print('  API docs:    http://localhost:8000/docs')
print('  MLflow UI:   http://localhost:5000')


---

## G.4 — Build, Run, and Debug

The commands below are run in a terminal (not in Colab).
Colab does not have Docker installed, but these are the exact commands
you would run on your local machine or a CI server.


In [ ]:
# G.4.1 -- Commonly used Docker commands (reference)

DOCKER_COMMANDS = {
    'Build the image': [
        'docker build -t salary-api:v1.0 .',
        '# -t  tag the image as salary-api version 1.0',
        '# .   use the Dockerfile in the current directory',
    ],
    'Run the container': [
        'docker run -d \\',
        '  -p 8000:8000 \\',
        '  -e MODEL_URI="runs:/abc123/model" \\',
        '  --name salary-api \\',
        '  salary-api:v1.0',
        '# -d  run in background (detached)',
        '# -p  map host port 8000 to container port 8000',
        '# -e  pass environment variables',
    ],
    'View logs': [
        'docker logs salary-api',
        'docker logs -f salary-api  # follow (like tail -f)',
    ],
    'Open a shell inside running container': [
        'docker exec -it salary-api /bin/bash',
    ],
    'Stop and remove container': [
        'docker stop salary-api',
        'docker rm salary-api',
    ],
    'List images and containers': [
        'docker images                    # all images',
        'docker ps                        # running containers',
        'docker ps -a                     # all containers (including stopped)',
    ],
    'Docker Compose': [
        'docker-compose up --build        # build and start all services',
        'docker-compose up -d             # start in background',
        'docker-compose logs -f           # follow logs from all services',
        'docker-compose down              # stop and remove containers',
        'docker-compose down -v           # also delete named volumes',
    ],
    'Push to Docker Hub': [
        'docker login',
        'docker tag salary-api:v1.0 yourusername/salary-api:v1.0',
        'docker push yourusername/salary-api:v1.0',
    ],
}

for section, commands in DOCKER_COMMANDS.items():
    print(f'# {section}')
    for cmd in commands:
        print(f'  {cmd}')
    print()


---

## G.5 — Keeping Images Small and Deploying to the Cloud

**Why image size matters:** smaller images pull faster, start faster,
cost less to store in a registry, and have a smaller attack surface.

**Techniques used in our Dockerfile:**

- **`python:3.11-slim`** instead of `python:3.11`: removes dev tools, saves ~800MB
- **Multi-stage build:** build dependencies are installed in stage 1 and only
  the compiled packages are copied to stage 2 — no compilers in the final image
- **`--no-cache-dir`** on pip: don't cache wheel files inside the image
- **`.dockerignore`:** prevents notebooks, test files, and large data files
  from being copied into the image context

**Cloud deployment options:**

| Platform | How to deploy | Best for |
|----------|--------------|----------|
| **Google Cloud Run** | `gcloud run deploy` | Serverless, auto-scales to zero |
| **AWS App Runner** | Push to ECR → deploy | Managed, no Kubernetes needed |
| **Azure Container Apps** | `az containerapp create` | Integrated with Azure ML |
| **Railway / Render** | Connect Docker Hub repo | Simplest for side projects |
| **Kubernetes (GKE/EKS)** | Helm chart or manifest | Large-scale, full control |

For a FastAPI salary prediction endpoint, **Cloud Run** is the recommended starting
point: zero infrastructure management, scales to zero when idle (no idle cost),
and deploys with a single command:

```bash
# Build and push to Google Container Registry
gcloud builds submit --tag gcr.io/YOUR_PROJECT/salary-api

# Deploy to Cloud Run
gcloud run deploy salary-api \\
  --image gcr.io/YOUR_PROJECT/salary-api \\
  --platform managed \\
  --region us-central1 \\
  --allow-unauthenticated \\
  --set-env-vars MODEL_URI=models:/so2025_salary_predictor/Production
```


---

## Appendix G Summary

### Key Takeaways

- **Containers solve environment reproducibility.** The same image runs identically
  on a developer laptop, a CI server, and a cloud VM.
- **Multi-stage builds** keep images small: use a `builder` stage with compilers
  and only copy the compiled output to the minimal runtime stage.
- **Layer caching** is the most important build performance technique.
  Always `COPY requirements.txt` and install dependencies *before* `COPY . .`
  so the slow install step is cached when only your code changes.
- **`.dockerignore`** is as important as `.gitignore`. Without it, notebooks,
  test data, and `.git/` directories bloat the image build context.
- **`docker-compose`** orchestrates multi-service stacks locally.
  The `depends_on: condition: service_healthy` pattern ensures services start
  in the right order.
- **`restart: unless-stopped`** makes containers self-healing — Docker automatically
  restarts them on crash or server reboot.
- **Cloud Run** is the lowest-friction path to production for a FastAPI ML endpoint:
  no Kubernetes, scales to zero, pay only for actual requests.

---

*End of Appendix G — Python for AI/ML*  
[![Back to TOC](https://img.shields.io/badge/Back_to-Table_of_Contents-1B3A5C?style=flat-square)](https://colab.research.google.com/github/timothy-watt/python-for-ai-ml/blob/main/Python_for_AIML_TOC.ipynb)
